In [1]:
pinecone='pcsk_33h2e2_KHZQcjsPQZ5Sbk4hqnYjCkvhoHzCT7wgTwFngrF4bQhbgeZhCtvJ6QDWQ5b43vs'

In [15]:
!pip -q install pinecone-client langchain_huggingface pinecone-text pinecone-notebooks langchain_community pinecone

In [7]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os 
from pinecone import Pinecone, ServerlessSpec
index_name="hybrid-search"
#initialize pinecone client
pc=Pinecone(api_key=pinecone)

In [12]:
#create index

if index_name  not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimensoin of dense vector since using Sent Transformers,
        metric='dotproduct',## sparse values supported in dot product
        spec=ServerlessSpec(cloud='aws', region="us-east-1"
    ))

In [13]:
index=pc.Index(index_name)
index

In [26]:
## vector embedding and sparse matrix 
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings
embedder=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
from pinecone_text.sparse import BM25Encoder

In [20]:
bm25_encoder=BM25Encoder().default()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
sentences=[
    
 'In 2023, i visited Paris',
    'in 2022 i visited New York',
    'in 2021 i visited New Orleans'

]

In [23]:
bm25_encoder.fit(sentences)

  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
bm25_encoder.dump("bm25_values.json")

In [28]:
retriever=PineconeHybridSearchRetriever(embeddings=embedder, sparse_encoder=bm25_encoder,index=index)

In [29]:
retriever.add_texts(
    sentences
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
retriever.invoke("which city did i visit the first?")

[Document(metadata={'score': 0.26246509}, page_content='in 2021 i visited New Orleans'),
 Document(metadata={'score': 0.240027189}, page_content='in 2022 i visited New York'),
 Document(metadata={'score': 0.235410631}, page_content='In 2023, i visited Paris')]